In [ ]:
using Pkg,Statistics,Random,Printf,GZip,Knet,Plots

In [ ]:
Range=3.0; # range of the x values

In [ ]:
Noise_std=0.15; # max noise/value

In [ ]:
# obligatory arguments listed before ";" while optional arguments come after ";".
function gen_noisy_gaussian(;range=1.0,noise=0.1)
#    x = sort(randn(len)*range) # randn(): normal distributed 
    x = collect(-Range:0.2:Range)
    
#    y = exp.(-x.^2).*(-noise*2*(rand(length(x)).-0.5).+1) # fractional noise
     y = exp.(-x.^2) + randn(length(x))*noise; # additive gaussian noise
    return (x,y)
end

In [ ]:
(x_train,y_train) = gen_noisy_gaussian(range=Range,noise=Noise_std);

In [ ]:
L=length(x_train); # number of training data points

In [ ]:
plot(x_train,[y_train,exp.(-x_train.^2)])

In [ ]:
# transpose for easier manipulation during training
x_train = permutedims(x_train)
y_train = permutedims(y_train)

In [ ]:
Layersize = 50; # number of neurons in a layer

In [ ]:
# output = 1
# input = 1
# hidden = 50
# batchsize = 1
# one layer: tahn.(w[hidden,input] * x[input,batchsize] .+ b[hidden,1])

In [ ]:
# initialize weights
w = [0.1*rand(Layersize,1),0.1*rand(Layersize,1),0.1*rand(1,Layersize),0.1*rand(1,1)]

In [ ]:
# dimensions:
# w[1]: (hidden x input) - input->hidden weights
# x: (input x batchsize) - input
# w[2]: (hidden x 1) - input->hidden bias
# w[3]: (hidden x output) - hidden-> output weights
# w[4]: (output x 1) - hidden->output bias

function loss(w,x,y)
    guesses = sum(w[3] * tanh.(w[1]*x.+w[2]) .+ w[4],dims=1) # w[1]=w, w[2]=w0, w[3]=w', w[4]=w0'
    return mean(abs2,y-guesses)
end

In [ ]:
# construct the gradient-calculating function
lossgradient = grad(loss)

In [ ]:
dw = lossgradient(w,[x_train[1]],[y_train[1]])  # dw has dimnensions of w
                                                # output is the gradient w.r.t. the corresponding weight

In [ ]:
function mytrain!(w;lr=0.1)
    for n=1:L
        dw = lossgradient(w,[x_train[n]],[y_train[n]]);
        for i = 1:length(w)
            for j = 1:length(w[i])
                w[i][j] -= lr*dw[i][j];
            end
        end
    end
    return w
end

In [ ]:
# collect weights after each epoch in an array (trajectory)
weights = [ deepcopy(mytrain!(w,lr=0.01)) for epoch=1:1000 ];  # copy only copies the top layer, does not descend.

In [ ]:
loss(w,x_train,y_train)

In [ ]:
xplot=collect(-Range:0.01:Range);

In [ ]:
y_pred = permutedims(sum(w[3] * tanh.(w[1]*permutedims(xplot).+w[2]) .+ w[4],dims=1));

In [ ]:
# plot the converged function and the initial gaussian
plot(xplot,[y_pred,exp.(-xplot.^2)])

In [ ]:
# add the training dataset
scatter!(x_train,y_train,leg=false)